In [3]:
import numpy as np
from PIL import Image

In [6]:
Image.MAX_IMAGE_PIXELS = 933120000
flagged_img = np.asarray(Image.open('../data/flags_applied/2021_10_03/wq.tif'))

In [8]:
flagged_img.shape

(9410, 27602)

We aim to train the Unet model.

Testing Image = "unflagged"

Testing Label = "flagged"

#Categories and pixel value

Land=0

invalid=253 - 255

valid=others

We are going to classify pixels into valid and invalid.
Masks provides the value 0 to 255(which means RGB has already converted into 2D_label).
You can check these labels' value like this.

e.g.,
```python
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 933120000
flagged_img = np.asarray(Image.open('../data/flags_applied/2021_10_03/wq.tif'))
labels = np.expand_dims(flagged_img, axis=0)
np.unique(labels)
```

On the other hand, RGB provides three values(each has 0 to 255 value).

